In [ ]:
import cv2
import easyocr
from pdf2image import convert_from_path
import os
import pandas as pd
import time

In [ ]:
ocr_df = pd.read_excel('ocr_df.xlsx', index_col=0)
ocr_df['easy_ocr'] = ''

In [ ]:
image_convert_time = 0
ocr_time = 0
images_converted = 0
reader = easyocr.Reader(['en'], gpu=True)  # load once only in memory.

for index, row in ocr_df.iterrows():
    print(index)
    directory = 'test_files'
    scratch_dir = 'scratch_dir'
    for filename in os.listdir(scratch_dir):
        f = os.path.join(scratch_dir, filename)
        os.remove(f)

    pdf_text = ''
    image_convert_start_time = time.time()
    images = convert_from_path(f'''{directory}/{row['item_filename']}.pdf''')
    for i in range(len(images)):
        images[i].save(f'{scratch_dir}\page{str(i)}.png', 'PNG')
        images_converted += 1
    image_convert_end_time = time.time()
    image_convert_time += (image_convert_end_time - image_convert_start_time)

    ocr_start_time = time.time()
    for i in range(len(images)):


        image = cv2.imread(f'{scratch_dir}\page{str(i)}.png')
        ocr_text = reader.readtext(image, detail=0)
        ocr_text = ' '.join(ocr_text)
        pdf_text = pdf_text + ocr_text
    ocr_end_time = time.time()
    ocr_time += (ocr_end_time - ocr_start_time)

    ocr_df.at[index, 'easy_ocr'] = pdf_text

In [ ]:
print(f'Total time to convert images: {image_convert_time}')
print(f'Total time to OCR: {ocr_time}')
print(f'Total images converted: {images_converted}')

In [ ]:
ocr_df.to_excel('ocr_df_2023_retests.xlsx')